In [2]:
    #imports padrão
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


In [ ]:
    #ler o CSV
sams = pd.read_csv('C:\\Users\\Renato\\Downloads\\Case Analytics Engineer Sam\'s Club\\CASE_PRATICO_SAMS_CLUB.csv')
sams

In [ ]:
# Agrupa 10 valores com a menor MARGEM

grouped_sams_lowest = sams.groupby(['item_id', 'item_descricao'])
agg_sams_lowest = grouped_sams_lowest.agg({'receita_bruta': 'sum', 'item_unidade': 'sum', 'margem': 'sum'})
menores_margens = agg_sams_lowest.sort_values(by='margem', ascending=True).reset_index().head(15)
menores_margens


In [ ]:
#agrupa os valores com a MAIOR MARGEM
grouped_sams = sams.groupby(['departamento','periodo', 'item_id', 'item_descricao'])
agg_sams = grouped_sams.agg({'receita_bruta': 'sum', 'item_unidade': 'sum', 'margem': 'sum'})
top10_agrupado = agg_sams.sort_values(by='margem', ascending=False).reset_index().head(30)

# cria a coluna de coeficiente de margem
marg_recbrut = top10_agrupado
marg_recbrut['Coeficiente'] = marg_recbrut['margem']/marg_recbrut['item_unidade']
marg_recbrut_tratado = marg_recbrut.sort_values(by='Coeficiente', ascending=False).reset_index().head(30)
marg_recbrut_tratado

In [ ]:
#agrupa os valores com a MAIOR MARGEM
grouped_sams = sams.groupby(['item_id', 'item_descricao'])
agg_sams = grouped_sams.agg({'receita_bruta': 'sum', 'item_unidade': 'sum', 'margem': 'sum'})
top10_agrupado = agg_sams.sort_values(by='margem', ascending=False).reset_index().head(15)

# cria a coluna de coeficiente de margem
marg_recbrut = top10_agrupado
marg_recbrut['Coeficiente'] = marg_recbrut['margem']/marg_recbrut['item_unidade']
marg_recbrut_tratado = marg_recbrut.sort_values(by='Coeficiente', ascending=False).reset_index().head(16)
marg_recbrut_tratado

In [ ]:

#PARA QUALQUER ITEM - APENAS TROQUE VALOR DE X PELO CODIGO DO ITEM
#Encontra por comparacao de tickets unicos os 15 items mais comprados junto com os 10 itens mais relevantes

X = 1215826 # DIGITE O CODIGO DO ITEM TOP 10

encont_tickets = sams[sams['item_id'] == X]['ticket'].unique()
filtered_encont_tickets = sams[sams['ticket'].isin(encont_tickets)]
vazio = pd.DataFrame()


#Encontrar os 15 itens mais comprados com o item escolhido do top5 os possíveis combos duplos
classificacao = filtered_encont_tickets.sort_values('item_unidade', ascending=False)
sams_simples = classificacao.drop(['socio', 'ticket', 'id_clube','periodo'], axis=1)
sams_simples_group = sams_simples.groupby(['item_id', 'item_descricao'])
sams_simples_agg = sams_simples_group.agg({'receita_bruta': 'sum', 'item_unidade': 'sum', 'margem': 'sum'})
sams_simples_tratado = sams_simples_agg.sort_values(by='item_unidade', ascending=False).reset_index().drop(0).head(15)
item_unico = sams.loc[sams['item_id'] == X, 'item_descricao'].values[0]
sams_simples_tratado['combo'] = item_unico + ' + ' + sams_simples_tratado['item_descricao']
sams_simples_tratado['Condicional'] = item_unico
sams_simples_tratado




In [ ]:

X = 1228155 # item_id do item
n_top_items = 3 # numero de itens no combo

# encontra tickets unicos com o item X
encont_tickets = sams[sams['item_id'] == X]['ticket'].unique()
filtered_encont_tickets = sams[sams['ticket'].isin(encont_tickets)]

# Cria a classificao consequente da escolha de X
classification = filtered_encont_tickets.sort_values('item_unidade', ascending=False)
sams_simples = classification.drop(['socio', 'ticket', 'id_clube','periodo'], axis=1)
sams_simples_group = sams_simples.groupby(['item_id', 'item_descricao'])
sams_simples_agg = sams_simples_group.agg({'receita_bruta': 'sum', 'item_unidade': 'sum', 'margem': 'sum'})
sams_simples_sorted = sams_simples_agg.sort_values(by='item_unidade', ascending=False).reset_index()

# Remove o item X
sams_simples_sorted = sams_simples_sorted[sams_simples_sorted['item_id'] != X]

# pega os top itens
top_items = sams_simples_sorted.head(n_top_items)

# pega a descricao do item X
main_item_desc = sams.loc[sams['item_id'] == X, 'item_descricao'].values[0]

# cria o combo custom
top_items_desc = ' + '.join(top_items['item_descricao'])
custom_combo = main_item_desc + ' + ' + top_items_desc

print(custom_combo)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

# Extraindo uma lista de transacoes do ticket x item_id
transactions = sams.groupby('ticket')['item_id'].apply(list).tolist()

# One-hot encoding
encoder = TransactionEncoder()
encoded_transactions = encoder.fit_transform(transactions)
df_transactions = pd.DataFrame(encoded_transactions, columns=encoder.columns_)

from mlxtend.frequent_patterns import apriori

# Ajuste min_support como for preciso para encontrar itemsets relevantes. O meu foi limitado pelo processamento do computador.
frequent_itemsets = apriori(df_transactions, min_support=0.01, use_colnames=True)

from mlxtend.frequent_patterns import association_rules

# Cria as regras, ajuste a precisão do min_threshold de acordo com a vontade. O meu ficou limitado pelo processamento do meu computador. 
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)


# garanta que o valor seja em string
X = 1215794  # The item of interest
rules['antecedents'] = rules['antecedents'].apply(lambda x: set(map(str, x)))

# se item_ids inteiros, entao os transforme em string se usar antecedents em string
filtered_rules = rules[rules['antecedents'] == {str(X)}]

# classifica pela metrica de interesse, nesse caso setamos 'confidence'
filtered_rules_sorted = filtered_rules.sort_values(by='confidence', ascending=False)

# Mostra as metricas resultantes
print(filtered_rules_sorted)
